In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import openslide
import os
import cv2
import PIL
from PIL import Image
from IPython.display import Image, display
from keras.applications.vgg16 import VGG16,preprocess_input
# Plotly for the interactive viewer (see last section)
import plotly.graph_objs as go
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model,load_model
from keras.applications.vgg16 import VGG16,preprocess_input
from keras.applications.resnet50 import ResNet50
from keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten,BatchNormalization,Activation
from keras.layers import GlobalMaxPooling2D
from keras.models import Model
from keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau
from keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
import gc
import matplotlib.pyplot as plt
%matplotlib inline
import skimage.io
from sklearn.model_selection import KFold
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.
import tensorflow as tf
from tensorflow.python.keras import backend as K
sess = K.get_session()

In [ ]:
from keras.callbacks.callbacks import ReduceLROnPlateau, EarlyStopping
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from keras.models import load_model
from keras.applications.xception import Xception
import time
import seaborn as sns
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

In [ ]:

from numpy.random import seed
seed(1)

In [ ]:
train=pd.read_csv('/kaggle/input/prostate-cancer-grade-assessment/train.csv')
train.head()
train_copy = train.copy()

In [ ]:
img=openslide.OpenSlide('/kaggle/input/prostate-cancer-grade-assessment/train_images/2fd1c7dc4a0f3a546a59717d8e9d28c3.tiff')
display(img.get_thumbnail(size=(512,512)))

In [ ]:
img.dimensions

In [ ]:
patch = img.read_region((18500,4100), 0, (256, 256))

# Display the image
display(patch)
# Close the opened slide after use
img.close()

In [ ]:
train['isup_grade'].value_counts()

The classes are imbalanced, more severe cases are underrepresented.

Let's start preparing the images for training.

In [ ]:
train.head()

In [ ]:
labels=[]
data=[]
data_dir='../input/panda-resized-train-data-512x512/train_images/train_images/'
for i in range(train.shape[0]):
    data.append(data_dir + train['image_id'].iloc[i]+'.png')
    labels.append(train['isup_grade'].iloc[i])
df=pd.DataFrame(data)
df.columns=['images']
df['isup_grade']=labels

In [ ]:
df.head()

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(df['images'],df['isup_grade'], test_size=0.20, random_state=42)


In [ ]:
train=pd.DataFrame(X_train)
train.columns=['images']
train['isup_grade']=y_train

validation=pd.DataFrame(X_val)
validation.columns=['images']
validation['isup_grade']=y_val

train['isup_grade']=train['isup_grade'].astype(str)
validation['isup_grade']=validation['isup_grade'].astype(str)

So the basic preprocessing I've done is:-
* Normalizing the images.
* Reshape the images to be of shape 224,224,3
* Basic image augmentation like rotation, flipping etc.

In [ ]:
#Image data generator generates varied images with the input data for better prediction for the models
#shear is avoided since the cancerous cells look like sheared normal cells

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,rotation_range=40,
    featurewise_center=True,
    featurewise_std_normalization=True,
    #zoom_range=[0.8, 1.2],        
    horizontal_flip=True, vertical_flip = True,
    brightness_range=[0.9, 1.1],
    width_shift_range=1.0,
    height_shift_range=1.0)#,
    #validation_split=0.1)


val_datagen=train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_dataframe(
    train,
    x_col='images',
    y_col='isup_grade',
    target_size=(224, 224),
    batch_size=32,
    shuffle = True,
    class_mode='categorical')

validation_generator = val_datagen.flow_from_dataframe(
    validation,
    x_col='images',
    y_col='isup_grade',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

In [ ]:
filenames = validation_generator.filenames
nb_samples = len(filenames)



In [ ]:
#y_true = validation_generator.classes



In [ ]:
#y_true

In [ ]:
#VGG16 model

In [ ]:
def vgg16_model( num_classes=None):

    model = VGG16(weights='/kaggle/input/keras-pretrained-models/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5', include_top=False, input_shape=(224, 224, 3))
    #x=Dropout(0.2)(model.output)
    x=Flatten()(model.output)
    #x =Dense(32, activation = 'relu')(x)
    output=Dense(num_classes,activation='softmax')(x)
    model=Model(model.input,output)
    return model

vgg_conv=vgg16_model(6)

In [ ]:
#VGG19 model-- Uncomment last line to run the model

In [ ]:
from keras.applications.vgg19 import VGG19
def vgg19_model(num_classes = None):
    vgg19_weights = '../input/vgg19/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5'
    #model = VGG19(weights='imagenet',include_top=False, input_shape=(224, 224, 3))
    model = VGG19(weights= vgg19_weights , include_top=False, input_shape=(224, 224, 3))
    x=Dropout(0.3)(model.output)
    x=Flatten()(x)
    x =Dense(32, activation = 'relu')(x)
    x =Dropout(0.2)(x)
    output=Dense(num_classes,activation='softmax')(x)
    model=Model(model.input,output)
    return model
vgg19_conv = vgg19_model(6)

In [ ]:
#InceptionV3 model- Uncomment last line to run the model

In [ ]:
from keras.applications.inception_v3 import InceptionV3
def InceptionV3_model(num_classes = None):
    InceptionV3_weights = '../input/inceptionv3/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
    #model = ResNet50(weights='imagenet', include_top = False, input_shape = (224,224,3))
    model = InceptionV3(weights= InceptionV3_weights, include_top=False, input_shape=(224, 224, 3))
    #model = InceptionV3(weights='imagenet', include_top = False, input_shape = (224,224,3))
    x=Dropout(0.3)(model.output)
    #x=Flatten()(model.output)
    #x =Dense(64, activation = 'relu')(model.output)
    #x = tf.compat.v1.keras.layers.GlobalAveragePooling2D()(model.output)
    x=Flatten()(x)
    #x =Dropout(0.2)(x)
    x =Dense(32, activation = 'relu')(x)
    x =Dropout(0.2)(x)
    output=Dense(num_classes,activation='softmax')(x)
    model=Model(model.input,output)
    return model
InceptionV3_conv = InceptionV3_model(6)

In [ ]:
#Resnet50 model- Uncomment last line to run the model

In [ ]:
!pip install tensorflow==2.1.0

In [ ]:
from keras.applications.resnet50 import ResNet50
def ResNet50_model(num_classes = None):
    ResNet_weights = '../input/keras-pretrained-models/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
    #model = ResNet50(weights='imagenet', include_top = False, input_shape = (224,224,3))
    model = ResNet50(weights= ResNet_weights, include_top=False, input_shape=(224, 224, 3))
    #x=Dropout(0.2)(model.output)
    #x = GlobalAveragePooling2D()(model.output)
    x=Flatten()(model.output)
    #x =Dropout(0.2)(x)
    x =Dense(16, activation = 'relu')(x)
    x =Dropout(0.2)(x)
    output=Dense(num_classes,activation='softmax')(x)
    model=Model(model.input,output)
    return model
ResNet50_conv = ResNet50_model(6)

In [ ]:
#InceptionV3_conv,ResNet50_conv

In [ ]:
from keras.applications import InceptionResNetV2
def InceptionResnet_model(num_classes = None):
    InceptionResnet_weights = '../input/keras-pretrained-models/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5'
    #model = inception_resnet_v2(weights='imagenet', include_top = False, input_shape = (224,224,3))
    model = InceptionResNetV2(weights= InceptionResnet_weights, include_top=False, input_shape=(224, 224, 3))
    #x=Dropout(0.2)(model.output)
    #x = GlobalAveragePooling2D()(model.output)
    x=Flatten()(model.output)
    #x =Dropout(0.2)(x)
    x =Dense(16, activation = 'relu')(x)
    x =Dropout(0.2)(x)
    output=Dense(num_classes,activation='softmax')(x)
    model=Model(model.input,output)
    return model
InceptionResnet_conv = InceptionResnet_model(6)

Let's take a look at the architecture.

In [ ]:
#Uncomment any one to run the corresponding summary

In [ ]:
#vgg_conv.summary()

In [ ]:
#vgg19_conv.summary()

In [ ]:

#InceptionV3_conv.summary()

In [ ]:
#ResNet50_conv.summary()

In [ ]:
#InceptionResnet_conv.summary()

In [ ]:
#Including batch Normalization

In [ ]:
def freeze(model):
    # freeze layers before 99
    for layer in model.layers[:99]:
        layer.trainable = False
        if layer.name.startswith('batch_normalization'):
            layer.trainable = True
        if layer.name.endswith('bn'):
            layer.trainable = True

    # unfreeze layers after 99
    for layer in model.layers[99:]:
        layer.trainable = True
        

        
        
        

In [ ]:
freeze(vgg_conv)
freeze(vgg19_conv)
freeze(InceptionV3_conv)
freeze(ResNet50_conv)
freeze(InceptionResnet_conv)

In [ ]:
#y_pred = np.argmax(InceptionV3_conv.predict_generator(validation_generator, steps= len(validation_generator)), axis=1)

In [ ]:
#kappa score is the metric we use since two types of evaluation is done on the data set 

#The kappa statistic.
#According to Cohen's original article, 
#values ≤ 0 as indicating no agreement and 0.01–0.20 as none to slight,
#0.21–0.40 as fair, 0.41– 0.60 as moderate,
#0.61–0.80 as substantial, 
#0.81–1.00 as almost perfect agreement.

In [ ]:
#def kappa_score(y_true, y_pred):
    
    #y_true=tf.math.argmax(y_true)
    #y_pred=tf.math.argmax(y_pred)
    #return tf.compat.v1.py_func(cohen_kappa_score,(y_true, y_pred),tf.double)
    #return tf.compat.v1.py_func(cohen_kappa_score,(y_true,y_pred),tf.double)
    #return (y_true,y_pred)

Hyperparameter Tuning 

In [ ]:
opt = Adam(lr= 0.0001)
vgg_conv.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])
vgg19_conv.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])
InceptionV3_conv.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])
ResNet50_conv.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])
InceptionResnet_conv.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])

In [ ]:
nb_epochs = 30
batch_size=16
#nb_train_steps = train.shape[0]//batch_size
#nb_val_steps=validation.shape[0]//batch_size
nb_train_steps = 128
nb_val_steps = 64
print("Number of training and validation steps: {} and {}".format(nb_train_steps,nb_val_steps))

In [ ]:
vgg_conv.fit_generator( train_generator,steps_per_epoch=nb_train_steps,epochs=30,validation_data=validation_generator,
validation_steps=128)


In [ ]:
#generator to activate the augmentation.
#callbacks = [ReduceLROnPlateau(monitor='val_loss', patience=1, verbose=1, factor=0.5),
             #EarlyStopping(monitor='val_loss', patience=3),
             #ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

In [ ]:
#tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs")

In [ ]:
vgg19_conv.fit_generator( train_generator,steps_per_epoch=nb_train_steps,epochs=nb_epochs,validation_data=validation_generator,validation_steps=nb_val_steps)

In [ ]:
history =InceptionV3_conv.fit_generator( train_generator,steps_per_epoch=nb_train_steps,epochs=nb_epochs,validation_data=validation_generator,
validation_steps=nb_val_steps)

In [ ]:

ResNet50_conv.fit_generator( train_generator,steps_per_epoch=nb_train_steps,epochs=30,validation_data=validation_generator,
validation_steps=nb_val_steps)

In [ ]:
history = InceptionResnet_conv.fit_generator( train_generator,steps_per_epoch=nb_train_steps,epochs=30,validation_data=validation_generator,
validation_steps=nb_val_steps)

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3, VGG19, ResNet50
from tensorflow.keras.applications.inception_v3 import preprocess_input as process_inception_v3
from tensorflow.keras.applications.vgg19 import preprocess_input as process_vgg19
from tensorflow.keras.applications.resnet50 import preprocess_input as process_resnet
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.optimizers import *

import cv2
import random
import itertools
import os

from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
### UTILITY FUNCTION TO LOAD BASE MODELS ###

def import_base_model(SHAPE):
    #InceptionV3_weights = '../input/inceptionv3/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
    inception_v3 =InceptionV3(weights= '../input/inceptionv3/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5',include_top = False, input_shape = SHAPE)
    #vgg19_weights = '../input/vgg19/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5'
    vgg19 = VGG19(weights = '../input/vgg19/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5', include_top = False, input_shape = SHAPE)
    #ResNet_weights = '../input/keras-pretrained-models/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
    resnet = ResNet50(weights = '../input/keras-pretrained-models/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5' , include_top = False, input_shape = SHAPE)

    #for layer in inception_v3.layers[:-4]:
        #layer.trainable = False

    #for layer in vgg19.layers[:-5]:
     #   layer.trainable = False

    #for layer in resnet.layers[:-10]:
     #   layer.trainable = False
        
    return inception_v3, vgg19, resnet

In [ ]:
### LOAD BASE MODELS ###
SHAPE=(224, 224, 3)
inception_v3, vgg19, resnet = import_base_model(SHAPE)

In [ ]:



inp = Input((224,224,3))

inception_v3_process = Lambda(process_inception_v3)(inp)
inception_v3 = inception_v3(inception_v3_process)
x_inception_v3 = GlobalMaxPool2D()(inception_v3)
x_inception_v3 = Dense(128, activation='relu')(x_inception_v3)

resnet_process = Lambda(process_resnet)(inp)
res_net = resnet(resnet_process)
x_resnet = GlobalMaxPool2D()(res_net)
x_resnet = Dense(128, activation='relu')(x_resnet)

vgg_19_process = Lambda(process_vgg19)(inp)
vgg_19 = vgg19(vgg_19_process)
x_vgg_19 = GlobalMaxPool2D()(vgg_19)
x_vgg_19 = Dense(128, activation='relu')(x_vgg_19)

x = Concatenate()([x_inception_v3, x_resnet, x_vgg_19])
out = Dense(6, activation='softmax')(x)

model = Model(inp, out)
model.compile(loss='categorical_crossentropy', optimizer=Nadam(lr=1e-4), metrics='accuracy')

es = EarlyStopping(monitor='val_accuracy', mode='auto', restore_best_weights=True, verbose=1, patience=7)
model.fit(train_generator, steps_per_epoch = train_generator.samples/train_generator.batch_size,
          epochs=50, validation_data=validation_generator, validation_steps = validation_generator.samples/validation_generator.batch_size, 
          callbacks=[es], verbose = 1)

In [ ]:
#history = InceptionResnet_conv.fit_generator( train_generator,steps_per_epoch=nb_train_steps,epochs=2,validation_data=validation_generator,
#validation_steps=nb_val_steps, callbacks = callbacks)

In [ ]:
def show_history(history):
    fig, ax = plt.subplots(1, 3, figsize=(15,5))
    ax[0].set_title('loss')
    ax[0].plot(history.epoch, history.history["loss"], label="Train loss")
    ax[0].plot(history.epoch, history.history["val_loss"], label="Validation loss")
    ax[1].set_title('kappa_keras')
    ax[1].plot(history.epoch, history.history["kappa_keras"], label="Train Quadratic Kappa score")
    ax[1].plot(history.epoch, history.history["val_kappa_keras"], label="Validation Quadratic Kappa score")
    ax[2].set_title('accuracy')
    ax[2].plot(history.epoch, history.history["accuracy"], label="Train acc")
    ax[2].plot(history.epoch, history.history["val_accuracy"], label="Validation accuracy")
    ax[0].legend()
    ax[1].legend()
    ax[2].legend()

In [ ]:
show_history(history)

In [ ]:
#vgg_baseline = Model.save('VGG16_Baseline.h5')  # creates a HDF5 file 'my_model.h5'
#InceptionV3_baseline =InceptionV3_conv.save('InceptionV3_Baseline.h5')  # creates a HDF5 file 'my_model.h5'
#Resnet50_baseline =ResNet50_conv.save('ResNet50_Baseline.h5')  # creates a HDF5 file 'my_model.h5'
#InceptionResnet_baseline =InceptionResnet_conv.save('InceptionResnet_Baseline.h5')  # creates a HDF5 file 'my_model.h5'

In [ ]:
#vgg_baseline_weights = Model.save_weights('vgg_baseline_weights.h5')
#InceptionV3_weights =InceptionV3_conv.save_weights('InceptionV3_weights.h5')  # creates a HDF5 file 'my_model.h5'
#Resnet50_weights =ResNet50_conv.save_weights('Resnet50_weights.h5')  # creates a HDF5 file 'my_model.h5'
#InceptionResnet_weights =InceptionResnet_conv.save_weights('InceptionResnet_weights.h5')  # creates a HDF5 file 'my_model.h5'

Inference


In [ ]:
os.listdir('.')

In [ ]:
#InceptionV3_conv.load_weights("best_model.h5")
#InceptionResnet_conv.load_weights("best_model.h5")

In [ ]:
def predict_submission(df, path, passes=1):
    
    df["image_path"] = [path+image_id+".tiff" for image_id in df["image_id"]]
    df["isup_grade"] = 0
    
    for idx, row in df.iterrows():
        prediction_per_pass = []
        for i in range(passes):
            model_input = np.array([get_random_samples(row.image_path)/255.])
            input_image1 = model_input[:,0,:,:]
            input_image2 = model_input[:,1,:,:]
            input_image3 = model_input[:,2,:,:]

            prediction = InceptionResnet_conv.predict([input_image1,input_image2,input_image3])
            prediction_per_pass.append(np.argmax(prediction))
            
        df.at[idx,"isup_grade"] = np.mean(prediction_per_pass)
    df = df.drop('image_path', 1)
    return df[["image_id","isup_grade"]]

In [ ]:
# submission code from https://www.kaggle.com/frlemarchand/high-res-samples-into-multi-input-cnn-keras
def predict_submission(df, path):
    
    df["image_path"] = [path+image_id+".tiff" for image_id in df["image_id"]]
    df["isup_grade"] = 0
    predictions = []
    for idx, row in df.iterrows():
        print(row.image_path)
        img=skimage.io.imread(str(row.image_path))
        img = cv2.resize(img, (224,224))
        img = cv2.resize(img, (224,224))
        img = img.astype(np.float32)/255.
        img=np.reshape(img,(1,224,224,3))
       
    
        prediction=InceptionResnet_conv.predict(img)
        predictions.append(np.argmax(prediction))
            
    df["isup_grade"] = predictions
    df = df.drop('image_path', 1)
    return df[["image_id","isup_grade"]]

In [ ]:
test_path = "../input/prostate-cancer-grade-assessment/test_images/"
submission_df = pd.read_csv("../input/prostate-cancer-grade-assessment/sample_submission.csv")

if os.path.exists(test_path):
    test_df = pd.read_csv("../input/prostate-cancer-grade-assessment/test.csv")
    submission_df = predict_submission(test_df, test_path)
else:
    print('submission csv not found')

    
del vgg16
submission_df.to_csv('submission.csv', index=False)
submission_df.head()